### manga downlader from mangalib.me

1) Get data from url
2) Make json from data
3) Make a download requests to make request file
4) Generate an output file

In [ ]:
MANGA_ALIAS="bungou-stray-dogs"
FROM=1
TO=1


LOAD_URL = "https://mangalib.me/"+MANGA_ALIAS+"?section=chapters&ui=9141522"
DEBUG_MODE = True
DATA_FOLDER_NAME = "./data/"+MANGA_ALIAS

In [ ]:
#1 Get data from url

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Setup Chrome options
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": "/home/vlusp/Downloads",  # Set your preferred download directory
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--auto-open-devtools-for-tabs")
options.add_argument("user-data-dir=selenium")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Setup driver
driver = webdriver.Chrome(options=options)
driver.maximize_window()

#delete window properties for cloudflare
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
	"source": """
		delete window.cdc_adoQpoasnfa76pfcZLmcfl_Array;
        delete window.cdc_adoQpoasnfa76pfcZLmcfl_JSON;
        delete window.cdc_adoQpoasnfa76pfcZLmcfl_Object;
        delete window.cdc_adoQpoasnfa76pfcZLmcfl_Promise;
        delete window.cdc_adoQpoasnfa76pfcZLmcfl_Proxy;
        delete window.cdc_adoQpoasnfa76pfcZLmcfl_Symbol;
	"""
})


# Open the page
driver.get(LOAD_URL)
data = driver.page_source
data


In [ ]:
#2 Make json from data

import json
import os

start = 'window.__DATA__ = '
end = 'window._SITE_COLOR_'

# slice data string to get json content
filtered_data = data[data.find(start)+len(start):data.find(end)].rstrip()
# remove last comma
filtered_data = filtered_data[:-1]
# convert to json
json_data = json.loads(filtered_data)
os.mkdir(DATA_FOLDER_NAME)
# write all data to file (debug mode only)
if DEBUG_MODE == True:
        with open(DATA_FOLDER_NAME+'/all.json', 'w') as f:
                json.dump(json_data, f)
# write to file
with open(DATA_FOLDER_NAME+'/manga.json', 'w') as f:
        json.dump(json_data['manga'], f)
# get chapters
chapters_data = json_data['chapters']['list']
# write to file
with open(DATA_FOLDER_NAME+'/chapters.json', 'w') as f:
        json.dump(chapters_data, f)
chapters_data

In [ ]:
#3 Make a download requests to make request file
for i in range(len(chapters_data)):
	item = chapters_data[i]
	driver.get(f'https://mangalib.me/download/{item["chapter_id"]}')
	pre = driver.find_element(By.TAG_NAME, 'pre').text
	image_data = json.loads(pre)
	item['urls'] = []
	for image in image_data['images']:
		item['urls'].append(f'{image_data["downloadServer"]}/manga/{MANGA_ALIAS}/chapters/{image_data["chapter"]["slug"]}/{image}')
	print(item)

with open(DATA_FOLDER_NAME + '/chapters.json', 'w') as f:
	json.dump(chapters_data, f)

In [ ]:
manga_data = json_data['manga']
manga_data['chapters'] = chapters_data
with open(DATA_FOLDER_NAME+'/manga.json', 'w') as f:
        json.dump(manga_data, f)

In [ ]:
#run this commands in terminal
#!/bin/bash
!make run MANGA_NAME=$MANGA_ALIAS VOL_FROM=$FROM VOL_TO=$TO WORKERS=8